In [149]:
import math
import pandas as pd
import numpy as np

In [150]:
df = pd.DataFrame(
    {
        'sku': [100, 200, 300, 400, 500, 600, 700],
        'gmv':   [400, 350, 500, 800, 0, -100, 200 ], 
        'price': [100, 70,  120, 200, 50, 43, -14],
        'stock': [3,   10,  5,  0, 100, 21, 53],

        
        
        # 'gmv':   [290, 300, 400, 350, 300, 0,  4, 0, 230], 
        # 'price': [100, 100, 100, 70,  120, 10, 4, 0, 230],
        # 'stock': [3,   3,   3,   10,  5,   1,  0, 0, 0],

    }
)
df

,sku,gmv,price,stock
0,100,400,100,3
1,200,350,70,10
2,300,500,120,5
3,400,800,200,0
4,500,0,50,100
5,600,-100,43,21
6,700,200,-14,53


In [157]:
df.dtypes

sku      int64
gmv      int64
price    int64
stock    int64
dtype: object

In [4]:
# corner cases - zero floor mixed?

def update_gmv(gmv, price, stock):
    """."""
    if stock == 0:
        return 0

    if price == 0:
        return 0
    
    if gmv == 0:
        return 0
    
    if (stock < 0) or (price < 0) or (gmv < 0):
        return 0
      
    if math.floor(gmv / price) > stock:
        return stock  * price
    
    if gmv % price != 0:
        return math.floor(gmv / price) * price

    return gmv


def limit_gmv(df: pd.DataFrame) -> pd.DataFrame:
    """."""
    # 1 version
    gmv_upd = df.apply(lambda x: update_gmv(x['gmv'], x['price'], x['stock']), axis=1)
    df2 = pd.DataFrame()
    df2['sku'] = df['sku']
    df2['gmv'] = gmv_upd
    df2[['price', 'stock']] = df[['price', 'stock']]
    df2['stock'] = df['stock']
    
    # 2 version
    # gmv_upd = df.apply(lambda x: update_gmv(x['gmv'], x['price'], x['stock']), axis=1)
    # df2 = df.copy()
    # df2.drop(columns=['gmv'], axis=1, inplace=True)
    # df2['gmv'] = gmv_upd
    # columns_order = ['sku', 'gmv', 'price', 'stock']
    # df2 = df2[columns_order]
    
    return df2

%timeit df2 = limit_gmv(df)

2.32 ms ± 285 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [138]:
df2

,sku,gmv,price,stock
0,100,300,100,3
1,200,350,70,10
2,300,480,120,5
3,400,0,200,0
4,500,0,50,100


In [164]:
df_arr = np.array(df)
df_arr

array([[100, 400, 100,   3],
       [200, 350,  70,  10],
       [300, 500, 120,   5],
       [400, 800, 200,   0],
       [500,   0,  50, 100]])

In [229]:
arr = df_arr[2]
# np.int32(np.floor(arr[1] / arr[2]))
arr

array([300, 500, 120,   5])

In [237]:
arr[1] % arr[2]

20

In [238]:
500 % 120

20

In [262]:
df

,sku,gmv,price,stock
0,100,400,100,3
1,200,350,70,10
2,300,500,120,5
3,400,800,200,0
4,500,0,50,100


In [264]:
arr[1] <= 0

False

In [21]:
def vect_gmv(arr):
    if arr[1] <= 0: return 0
    if arr[2] < 0: return 0
    if arr[3] <= 0: return 0

    if arr[1] > (arr[2] * arr[3]):
        return arr[2] * arr[3]

    temp = np.int32(np.floor(arr[1] / arr[2]))

    # if arr[1] % arr[2] != 0:
    #     return math.floor(arr[1] / arr[2]) * arr[2]
    
    return arr[1]


def limit_gmv(df: pd.DataFrame) -> pd.DataFrame:
    """."""
    df_arr = np.array(df)
    gmv_upd = np.apply_along_axis(func1d=vect_gmv, axis=1, arr=df_arr)

    return pd.DataFrame(
        {
            'sku': df_arr[:, 0], 
            'gmv': np.float64(gmv_upd), 
            'price': np.float64(df_arr[:, 2]), 
            'stock': df_arr[:, 3],
        }
    )

# %timeit  
# np.apply_along_axis(func1d=vect_gmv, axis=1, arr=df_arr)

# %timeit df2 = limit_gmv(df)
df2 = limit_gmv(df)
df2

,sku,gmv,price,stock
0,100,300.0,100.0,3
1,200,350.0,70.0,10
2,300,500.0,120.0,5
3,400,0.0,200.0,0
4,500,0.0,50.0,100


In [22]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   sku     5 non-null      int64  
 1   gmv     5 non-null      float64
 2   price   5 non-null      float64
 3   stock   5 non-null      int64  
dtypes: float64(2), int64(2)
memory usage: 288.0 bytes


In [21]:
def vect_gmv(arr):
    temp = np.clip(arr, 0, arr[2] * arr[3])

    internal = np.clip(
        temp,
        
    )
    
    temp = np.int32(np.floor(arr[1] / arr[2]))

  
    return arr[1]


def limit_gmv(df: pd.DataFrame) -> pd.DataFrame:
    """."""
    df_arr = np.array(df)
    gmv_upd = np.apply_along_axis(func1d=vect_gmv, axis=1, arr=df_arr)

    return pd.DataFrame(
        {
            'sku': df_arr[:, 0], 
            'gmv': np.float64(gmv_upd), 
            'price': np.float64(df_arr[:, 2]), 
            'stock': df_arr[:, 3],
        }
    )

# %timeit  
# np.apply_along_axis(func1d=vect_gmv, axis=1, arr=df_arr)

# %timeit df2 = limit_gmv(df)
df2 = limit_gmv(df)
df2

,sku,gmv,price,stock
0,100,300.0,100.0,3
1,200,350.0,70.0,10
2,300,500.0,120.0,5
3,400,0.0,200.0,0
4,500,0.0,50.0,100


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   sku     5 non-null      int64  
 1   gmv     5 non-null      int64  
 2   price   5 non-null      float64
 3   stock   5 non-null      int64  
dtypes: float64(1), int64(3)
memory usage: 288.0 bytes


In [15]:
df2['price'] = df2['price'].astype('float')

In [249]:
df_arr = np.array(df)
df_arr

array([[100, 400, 100,   3],
       [200, 350,  70,  10],
       [300, 500, 120,   5],
       [400, 800, 200,   0],
       [500,   0,  50, 100]])

In [245]:
arr[2] * arr[3]

600

In [211]:
df_arr = np.array(df)
df_arr

array([[100, 400, 100,   3],
       [200, 350,  70,  10],
       [300, 500, 120,   5],
       [400, 800, 200,   0],
       [500,   0,  50, 100]])

In [216]:
# df_arr[:, 0]

temp = pd.DataFrame(
    {
        'sku': df_arr[:, 0]
    }
)
temp



,sku
0,100
1,200
2,300
3,400
4,500


In [165]:
b = np.array([[1,2,3], [4,5,6], [7,8,9]])
pd.DataFrame(b)

,0,1,2
0,1,2,3
1,4,5,6
2,7,8,9


In [181]:
b

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [167]:
def my_func(a):
    return (a[0] + a[-1]) * 0.5
np.apply_along_axis(my_func, 0, b)

array([4., 5., 6.])

In [124]:
arr = df['sku']
arrgmv = df['gmv']

temp = pd.DataFrame()
temp['sku'] = arr
temp['gmv'] = arrgmv
temp

,sku,gmv
0,100,400
1,200,350
2,300,500
3,400,800
4,500,0


In [125]:
temp

,sku,gmv
0,100,400
1,200,350
2,300,500
3,400,800
4,500,0


In [260]:
np.where(
    (
        (df_arr[:, 1] <= 0) or (df_arr[:, 2] <= 0) or (df_arr[:, 3] <= 0)
    ), 
    0,
    df_arr[:, 3],
)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [114]:
type(df.apply(lambda x: update_gmv(x['gmv'], x['price'], x['stock']), axis=1))

pandas.core.series.Series

In [75]:
temp = df
temp

,gmv,price,stock
0,290,100,3
1,300,100,3
2,400,100,3
3,350,70,10
4,300,120,5
5,0,10,1
6,4,4,0
7,0,0,0
8,230,230,0


In [76]:
df['temp'] = df['stock']
df

,gmv,price,stock,temp
0,290,100,3,3
1,300,100,3,3
2,400,100,3,3
3,350,70,10,10
4,300,120,5,5
5,0,10,1,1
6,4,4,0,0
7,0,0,0,0
8,230,230,0,0


In [77]:
temp

,gmv,price,stock,temp
0,290,100,3,3
1,300,100,3,3
2,400,100,3,3
3,350,70,10,10
4,300,120,5,5
5,0,10,1,1
6,4,4,0,0
7,0,0,0,0
8,230,230,0,0


In [21]:
df2 = limit_gmv(df)
df2

,price,stock,gmv_upd
0,100,3,300
1,70,10,350
2,120,5,300
3,10,1,0
4,4,0,0
5,0,0,0


In [13]:
from random import random
import numpy as np

In [43]:
df3 = pd.DataFrame(
    {
        'gmv': np.random.randint(low=-100, high=1500, size=50), 
        'price': np.random.randint(low=-100, high=1500, size=50), 
        'stock': np.random.randint(low=-100, high=1500, size=50), 
    }
)
df3

,gmv,price,stock
0,451,1437,175
1,1328,1266,966
2,932,-63,406
3,1007,442,1155
4,1497,1297,1224
5,970,1322,1332
6,858,-3,119
7,959,529,140
8,1006,543,601
9,33,716,1290


In [ ]:
df['stock_check'] = df4[]

In [45]:
df4 = limit_gmv(df3)
df4['stock_check'] = df4['gmv'] / df4['price']

df4

    # if math.floor(gmv / price) > stock:
    #     return stock * price
    # return gmv

# price	stock	gmv	stock_check
# 1437	175	451	0.313848

,price,stock,gmv,stock_check
0,1437,175,451,0.313848
1,1266,966,1328,1.048973
2,-63,406,0,-0.000000
3,442,1155,1007,2.278281
4,1297,1224,1497,1.154202
5,1322,1332,970,0.733737
6,-3,119,0,-0.000000
7,529,140,959,1.812854
8,543,601,1006,1.852670
9,716,1290,33,0.046089


## numpy one fuction solution

In [ ]:
def limit_gmv(df: pd.DataFrame) -> pd.DataFrame:
    # 
    # 
    pass

In [116]:
df

,sku,gmv,price,stock
0,100,400,100,3
1,200,350,70,10
2,300,500,120,5
3,400,800,200,0
4,500,0,50,100
5,600,-100,43,21
6,700,200,-14,53


In [132]:
df_arr = np.array(df)
df_arr

array([[ 100,  400,  100,    3],
       [ 200,  350,   70,   10],
       [ 300,  500,  120,    5],
       [ 400,  800,  200,    0],
       [ 500,    0,   50,  100],
       [ 600, -100,   43,   21],
       [ 700,  200,  -14,   53]])

In [133]:
df_arr[:, 1] = np.where(
    df_arr[:, 3] <= 0, 
    0, 
    df_arr[:, 1]
)

In [134]:
df_arr[df_arr[:, 1] > 0]

array([[100, 400, 100,   3],
       [200, 350,  70,  10],
       [300, 500, 120,   5],
       [700, 200, -14,  53]])

In [135]:
df_arr = np.array(df)
df_arr



df_arr[:, 1] = np.where(
    df_arr[:, 3] <= 0, 
    0, 
    df_arr[:, 1]
)

df_arr[:, 1] = np.where(
    df_arr[:, 2] <= 0, 
    0, 
    df_arr[:, 1]
)

df_arr[:, 1] = np.where(
    df_arr[:, 1] < 0, 
    0, 
    df_arr[:, 1]
)

df_arr

array([[100, 400, 100,   3],
       [200, 350,  70,  10],
       [300, 500, 120,   5],
       [400,   0, 200,   0],
       [500,   0,  50, 100],
       [600,   0,  43,  21],
       [700,   0, -14,  53]])

In [136]:
df_arr[df_arr[:, 1] > 0]

array([[100, 400, 100,   3],
       [200, 350,  70,  10],
       [300, 500, 120,   5]])

In [138]:
max_arr = df_arr[:, 3] * df_arr[:, 2]
max_arr

array([ 300,  700,  600,    0, 5000,  903, -742])

In [163]:
df = pd.DataFrame(
    {
        'sku': [100, 200, 300, 400, 500, 600, 700],
        'gmv':   [400, 350, 500, 800, 0, -100, 200 ], 
        'price': [100, 70,  120, 200, 50, 43, -14],
        'stock': [3,   10,  5,  0, 100, 21, 53],

        
        
        # 'gmv':   [290, 300, 400, 350, 300, 0,  4, 0, 230], 
        # 'price': [100, 100, 100, 70,  120, 10, 4, 0, 230],
        # 'stock': [3,   3,   3,   10,  5,   1,  0, 0, 0],

    }
)
df

,sku,gmv,price,stock
0,100,400,100,3
1,200,350,70,10
2,300,500,120,5
3,400,800,200,0
4,500,0,50,100
5,600,-100,43,21
6,700,200,-14,53


In [164]:
df.dtypes

sku      int64
gmv      int64
price    int64
stock    int64
dtype: object

In [237]:
def limit_gmv(df: pd.DataFrame) -> pd.DataFrame:
    """."""
    df_arr = np.array(df)

    df_arr[:, 1] = np.where(
        df_arr[:, 3] <= 0,
        0,
        df_arr[:, 1]
    )

    df_arr[:, 1] = np.where(
        df_arr[:, 2] <= 0,
        0,
        df_arr[:, 1]
    )

    df_arr[:, 1] = np.where(
        df_arr[:, 1] < 0,
        0,
        df_arr[:, 1]
    )

    df_arr[:, 1] = np.where(
        np.floor(df_arr[:, 1] / df_arr[:, 2]) > df_arr[:, 3],
        df_arr[:, 2] * df_arr[:, 3],
        df_arr[:, 1]
    )

    df_arr[:, 1]  = np.where(
        df_arr[:, 1]  %  df_arr[:, 2] != 0,
        np.floor(df_arr[:, 1] / df_arr[:, 2]) * df_arr[:, 2],
        df_arr[:, 1]
    )

    return pd.DataFrame(
        {
            'sku': df_arr[:, 0],
            'gmv': np.float64(df_arr[:, 1]),
            'price': np.float64(df_arr[:, 2]),
            'stock': df_arr[:, 3],
        }
    )

# %timeit limit_gmv(df)
# limit_gmv(df)
df2 = limit_gmv(df)
df2

,sku,gmv,price,stock
0,100,300.0,100.0,3
1,200,350.0,70.0,10
2,300,480.0,120.0,5
3,400,0.0,200.0,0
4,500,0.0,50.0,100
5,600,0.0,43.0,21
6,700,0.0,-14.0,53


In [238]:
df2.dtypes

sku        int64
gmv      float64
price    float64
stock      int64
dtype: object

In [240]:

def limit_gmv(df: pd.DataFrame) -> pd.DataFrame:
    """
    Update gmv in data frame by checking counts stored.

    Parameters
    ----------
    df : pd.DataFrame
        Starter dataframe

    Returns
    -------
    pd.DataFrame
        New data frame with metric updated

    """
    df_arr = np.array(df)

    df_arr[:, 1] = np.where(
        df_arr[:, 3] <= 0,
        0,
        df_arr[:, 1]
    )

    df_arr[:, 1] = np.where(
        df_arr[:, 2] <= 0,
        0,
        df_arr[:, 1]
    )

    df_arr[:, 1] = np.where(
        df_arr[:, 1] < 0,
        0,
        df_arr[:, 1]
    )

    df_arr[:, 1] = np.where(
        np.floor(df_arr[:, 1] / df_arr[:, 2]) > df_arr[:, 3],
        df_arr[:, 2] * df_arr[:, 3],
        df_arr[:, 1]
    )

    df_arr[:, 1]  = np.where(
        df_arr[:, 1] %  df_arr[:, 2] != 0,
        np.floor(df_arr[:, 1] / df_arr[:, 2]) * df_arr[:, 2],
        df_arr[:, 1]
    )

    return pd.DataFrame(
        {
            'sku': df_arr[:, 0],
            'gmv': np.float64(df_arr[:, 1]),
            'price': np.float64(df_arr[:, 2]),
            'stock': df_arr[:, 3],
        }
    )

#     result = pd.DataFrame(
#         {
#             'sku': df_arr[:, 0],
#             'gmv': df_arr[:, 1],
#             'price': df_arr[:, 2],
#             'stock': df_arr[:, 3],
#         }
#     )

#     result['sku'] = result['sku'].astype('int32')
#     result['gmv'] = result['gmv'].astype('float32')
#     result['price'] = result['price'].astype('float32')
#     result['stock'] = result['stock'].astype('int32')
    
#     return result
                                         


In [241]:
df2 = limit_gmv(df)
df2.dtypes

sku        int64
gmv      float64
price    float64
stock      int64
dtype: object

In [230]:
df2

,sku,gmv,price,stock
0,100,300,100,3
1,200,350,70,10
2,300,480,120,5
3,400,0,200,0
4,500,0,50,100
5,600,0,43,21
6,700,0,-14,53


In [169]:
f16 = np.float16("0.1")
f32 = np.float32(f16)

f32

0.099975586

In [170]:
np.int32(4.21)

4

In [179]:
test = pd.DataFrame(
    
    {
        'int': [42],
        'float': [42.0],
    }
)

test.dtypes

int        int64
float    float64
dtype: object

In [184]:
df_arr = np.array(df)


np.int32(df_arr[:, 1]),

(array([ 400,  350,  500,  800,    0, -100,  200]),)

In [200]:
df2.describe()

,sku,gmv,price,stock
count,7.00000,7.000000,7.000000,7.000000
mean,400.00000,161.428571,81.285713,27.428571
std,216.02469,208.360951,67.815224,36.782501
min,100.00000,0.000000,-14.000000,0.000000
25%,250.00000,0.000000,46.500000,4.000000
50%,400.00000,0.000000,70.000000,10.000000
75%,550.00000,325.000000,110.000000,37.000000
max,700.00000,480.000000,200.000000,100.000000


In [201]:
df.dtypes

sku      int64
gmv      int64
price    int64
stock    int64
dtype: object

In [208]:
df2.dtypes

sku        int64
gmv      float64
price    float32
stock      int64
dtype: object

In [212]:
df2['sku'].astype('float')

0    100.0
1    200.0
2    300.0
3    400.0
4    500.0
5    600.0
6    700.0
Name: sku, dtype: float64

In [213]:
df2.dtypes

sku        int64
gmv      float64
price    float32
stock      int64
dtype: object

In [214]:
df2['sku'] = df2['sku'].astype('float')
df2.dtypes

sku      float64
gmv      float64
price    float32
stock      int64
dtype: object

In [217]:
df2['stock'].astype('float', copy=True)
df2.dtypes

sku      float64
gmv      float64
price    float32
stock      int64
dtype: object

In [224]:
df2

,sku,gmv,price,stock
0,100.0,300.0,100.0,3
1,200.0,350.0,70.0,10
2,300.0,480.0,120.0,5
3,400.0,0.0,200.0,0
4,500.0,0.0,50.0,100
5,600.0,0.0,43.0,21
6,700.0,0.0,-14.0,53


In [223]:
test = np.array(df2)
test

array([[100., 300., 100.,   3.],
       [200., 350.,  70.,  10.],
       [300., 480., 120.,   5.],
       [400.,   0., 200.,   0.],
       [500.,   0.,  50., 100.],
       [600.,   0.,  43.,  21.],
       [700.,   0., -14.,  53.]])

In [234]:
foo = pd.DataFrame(
    {
        'a': pd.Series(df_arr[:, 1], dtype=np.dtype("float")),
        'b': pd.Series(df_arr[:, 1], dtype=np.dtype("float"))
    }
)
foo

,a,b
0,400.0,400.0
1,350.0,350.0
2,500.0,500.0
3,800.0,800.0
4,0.0,0.0
5,-100.0,-100.0
6,200.0,200.0


In [232]:
foo = pd.DataFrame(
    {
        'a': df_arr[:, 1],
    }
)
foo

,a
0,400
1,350
2,500
3,800
4,0
5,-100
6,200


In [233]:
foo.dtypes

a    int64
dtype: object

In [239]:
df2

,sku,gmv,price,stock
0,100,300.0,100.0,3
1,200,350.0,70.0,10
2,300,480.0,120.0,5
3,400,0.0,200.0,0
4,500,0.0,50.0,100
5,600,0.0,43.0,21
6,700,0.0,-14.0,53


In [242]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   sku     7 non-null      int64
 1   gmv     7 non-null      int64
 2   price   7 non-null      int64
 3   stock   7 non-null      int64
dtypes: int64(4)
memory usage: 352.0 bytes


In [243]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   sku     7 non-null      int64  
 1   gmv     7 non-null      float64
 2   price   7 non-null      float64
 3   stock   7 non-null      int64  
dtypes: float64(2), int64(2)
memory usage: 352.0 bytes


In [252]:
df

,sku,gmv,price,stock
0,100,400,100,3
1,200,350,70,10
2,300,500,120,5
3,400,800,200,0
4,500,0,50,100
5,600,-100,43,21
6,700,200,-14,53


In [244]:
df_arr = np.array(df)

#     df_arr[:, 1] = np.where(
#         df_arr[:, 3] <= 0,
#         0,
#         df_arr[:, 1]
#     )

In [279]:
gmv = df_arr[:, 1]
price = df_arr[:, 2]
stock = df_arr[:, 3]

In [285]:
first = np.clip(
    gmv, 
    gmv,
    stock * price, 
)
first

array([ 300,  350,  500,    0,    0, -100, -742])

In [290]:
sec = np.clip(
    first,
    a_min = 0,
    a_max = np.floor(first / price) * price,   
)
sec

array([ 300.,  350.,  480.,    0.,    0., -129., -742.])

In [289]:
third = np.clip(
    sec,
    a_min = 0,
    a_max = sec,   
)
third

array([ 300.,  350.,  480.,    0.,    0., -129., -742.])

In [292]:
np.where(sec < 0, 0, sec)

array([300., 350., 480.,   0.,   0.,   0.,   0.])

In [293]:
df_arr = np.array(df)


first = np.clip(
    df_arr[:, 1], 
    df_arr[:, 1],
    df_arr[:, 2] * df_arr[:, 3], 
)

sec = np.clip(
    first,
    a_min = 0,
    a_max = np.floor(first / df_arr[:, 2]) * df_arr[:, 2],   
)

result = np.where(sec < 0, 0, sec)
result


array([300., 350., 480.,   0.,   0.,   0.,   0.])

## version on clip base

In [301]:
def limit_gmv(df: pd.DataFrame) -> pd.DataFrame:
    """
    Update gmv in data frame by checking counts stored.

    Parameters
    ----------
    df : pd.DataFrame
        Starter dataframe

    Returns
    -------
    pd.DataFrame
        New data frame with metric updated

    """
    df_arr = np.array(df)
    first = np.clip(
        df_arr[:, 1], 
        df_arr[:, 1],
        df_arr[:, 2] * df_arr[:, 3], 
    )
    sec = np.clip(
        first,
        a_min = 0,
        a_max = np.floor(first / df_arr[:, 2]) * df_arr[:, 2],   
    )

    result = np.where(sec < 0, 0, sec)


    return pd.DataFrame(
        {
            'sku': np.int64(df_arr[:, 0]),
            'gmv': np.float64(result),
            'price': np.float64(df_arr[:, 2]),
            'stock': np.int64(df_arr[:, 3]),
        }
    )
%timeit limit_gmv(df)

345 µs ± 2.03 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [302]:
df2 = limit_gmv(df)
df2

,sku,gmv,price,stock
0,100,300.0,100.0,3
1,200,350.0,70.0,10
2,300,480.0,120.0,5
3,400,0.0,200.0,0
4,500,0.0,50.0,100
5,600,0.0,43.0,21
6,700,0.0,-14.0,53


In [303]:
df2.dtypes

sku        int64
gmv      float64
price    float64
stock      int64
dtype: object

In [294]:
df2

,sku,gmv,price,stock
0,100,300.0,100.0,3
1,200,350.0,70.0,10
2,300,480.0,120.0,5
3,400,0.0,200.0,0
4,500,0.0,50.0,100
5,600,0.0,43.0,21
6,700,0.0,-14.0,53


In [273]:
sec = np.clip(
    sec,
    a_min = 0,
    a_max = np.floor(gmv / price) * price,   
)
sec

array([ 300.,  350.,  480.,    0.,    0., -129.,    0.])

In [260]:
np.floor(gmv / price) * price

array([ 400.,  350.,  480.,  800.,    0., -129.,  210.])

In [257]:
df2

,sku,gmv,price,stock
0,100,300.0,100.0,3
1,200,350.0,70.0,10
2,300,480.0,120.0,5
3,400,0.0,200.0,0
4,500,0.0,50.0,100
5,600,0.0,43.0,21
6,700,0.0,-14.0,53
